# Dividend Calculation Tests
Tue 4 Sep 2018 - Sat 8 Sep 2018

In [1]:
import numpy as np
import pandas as pd

## Create DataFrames with test data

Test negative amounts as well -- the patronage calculations should work the same way if people withdraw money.

Also, rename equity types from 'preferred', 'membership', and 'equity' to 'preferred', 'membership', and 'other', and use the term 'equity' to refer to all three types.

In [2]:
contributions_df = pd.DataFrame([
    [1, "1/3/2017", 125, 'membership'],
    [2, "1/31/2017", 400, 'other'],
    [1, "2/20/2017", 5000, 'preferred'],
    [3, "3/15/2017", -2000, 'preferred'],
    [1, "4/13/2017", 500, 'other'],
    [2, "4/13/2017", 7500, 'preferred'],
    [4, "6/24/2017", 125, 'mebership'],
    [5, "7/17/2017", 3000, "preferred"],
    [6, "8/22/2017", -700, 'other'],
    [4, "10/18/2017", 20000, 'preferred'],
    [6, "11/11/2017", 1500, 'preferred']
], columns = ['member_id','date','amount','type'])
contributions_df

,member_id,date,amount,type
0,1,1/3/2017,125,membership
1,2,1/31/2017,400,other
2,1,2/20/2017,5000,preferred
3,3,3/15/2017,-2000,preferred
4,1,4/13/2017,500,other
5,2,4/13/2017,7500,preferred
6,4,6/24/2017,125,mebership
7,5,7/17/2017,3000,preferred
8,6,8/22/2017,-700,other
9,4,10/18/2017,20000,preferred


Add new members and new columns to reflect new equity naming convention.

In [3]:
prev_equity_df = pd.DataFrame([
    [1, 'Alice', 0, 0, 0],
    [2, 'Bob', 125, 0, 0],
    [3, 'Candice', 125, 3000, 200],
    [4, 'Darwin', 0, 0, 0],
    [5, 'Eve', 125, 1000, 100],
    [6, 'Federico', 125, 0, 700]
], columns = ['member_id', 'name', 'membership', 'preferred', 'other'])
prev_equity_df['equity'] = prev_equity_df['membership'] + prev_equity_df['preferred'] + prev_equity_df['other']
prev_equity_df

,member_id,name,membership,preferred,other,equity
0,1,Alice,0,0,0,0
1,2,Bob,125,0,0,125
2,3,Candice,125,3000,200,3325
3,4,Darwin,0,0,0,0
4,5,Eve,125,1000,100,1225
5,6,Federico,125,0,700,825


## Collect functions from patronage notebook

And edit them to reflect the new equity naming conventions for the input DataFrames.

In [4]:
def fraction_year_remaining(date):
    """Computes the fraction of the year remaining from a given date.
    """
    date = pd.to_datetime(date)
    offset = pd.tseries.offsets.YearBegin()
    year = date - offset
    next_year = date + offset
    #print(offset, year, next_year)
    return (next_year - date).days / (next_year - year).days

def compute_transaction_patronage(contributions_df):
    """Compute patronage for each transaction based on amount and fraction of year remaining.
    """
    contributions_df = contributions_df.copy()
    patronage = contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)
    contributions_df['patronage'] = patronage
    return contributions_df

def compute_new_patronage(contributions_df):
    """Compute each member's patronage from new contributions for the current year.
    """
    contributions_df = compute_transaction_patronage(contributions_df)
    return contributions_df[['member_id','patronage']].groupby(by='member_id').sum()

def compute_patronage(prev_equity_df, contributions_df):
    """Compute total patronage for each member from new contributions for the current year
        and existing equity from previous years.
    """
    patronage_df = prev_equity_df.set_index('member_id')[['name', 'equity']]
    patronage_df.rename(columns={'equity': 'old_patronage'}, inplace=True)
    
#     patronage_df['old_patronage'] = patronage_df['equity'] + patronage_df['preferred']
    patronage_df['new_patronage'] = compute_new_patronage(contributions_df)['patronage']
    patronage_df['patronage'] = patronage_df['old_patronage'] + patronage_df['new_patronage']
    
    patronage_df['proportionate_patronage'] = patronage_df['patronage'] / patronage_df['patronage'].sum()
    
    return patronage_df#[['name', 'old_patronage', 'new_patronage', 'patronage', 'proportionate_patronage']]

In [5]:
patronage_df = compute_patronage(prev_equity_df, contributions_df)
patronage_df

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,0,4799.657534,4799.657534,0.240185
2,Bob,125,5771.232877,5896.232877,0.295060
3,Candice,3325,-1600.000000,1725.000000,0.086323
4,Darwin,0,4175.000000,4175.000000,0.208926
5,Eve,1225,1380.821918,2605.821918,0.130401
6,Federico,825,-43.561644,781.438356,0.039105


In [6]:
compute_transaction_patronage(contributions_df)

,member_id,date,amount,type,patronage
0,1,1/3/2017,125,membership,124.315068
1,2,1/31/2017,400,other,367.123288
2,1,2/20/2017,5000,preferred,4315.068493
3,3,3/15/2017,-2000,preferred,-1600.000000
4,1,4/13/2017,500,other,360.273973
5,2,4/13/2017,7500,preferred,5404.109589
6,4,6/24/2017,125,mebership,65.410959
7,5,7/17/2017,3000,preferred,1380.821918
8,6,8/22/2017,-700,other,-253.150685
9,4,10/18/2017,20000,preferred,4109.589041


## Write and test function to compute patronage dividends

In [7]:
#Set the profit for the year
profit = 1874.37 #1,981.18 #=Actual profit for 2017
profit

1874.37

In [8]:
np.floor(100*profit/2)/100 + np.ceil(100*profit/2)/100

1874.3699999999999

In [9]:
profit/2

937.185

In [10]:
np.round(profit/2, 2)

937.17999999999995

In [11]:
2*np.round(profit/2, 2)

1874.3599999999999

In [12]:
def compute_indiv_collective_profit(profit, percent_individual):
    """Compute the individual and collective net income from the total net income.
    """
    indiv_profit = np.round(profit * percent_individual/100, 2)
    collective_profit = profit - indiv_profit
    return indiv_profit, collective_profit

def compute_dividends(patronage_df, profit, percent_individual=50):
    """Compute each member's dividend based on patronage for the year.
    """
#     indiv_profit, collective_profit = compute_indiv_collective_profit(profit, percent_individual)
    dividend_df = patronage_df[['name', 'proportionate_patronage']].copy()
    
    #Compute individual patronage allocations
    dividend_df['dividend'] = np.round(
        dividend_df['proportionate_patronage'] * profit * percent_individual/100, 2)
    
    # To account for rounding amounts to the nearest cent, we add up the individual dividends
    # to get the actual amount allocated to individual net income. Then we subtract this amount
    # from the total profit to get the collective net income.
    indiv_profit = dividend_df['dividend'].sum()
    collective_profit = profit - indiv_profit
    
    # We reserve member_id=0 for the collective account (or we could simply use names as keys)
    dividend_df.loc[0] = pd.Series({
        'name': 'CollectiveAcct',
        'proportionate_patronage': collective_profit / indiv_profit,
        'dividend': collective_profit
    })
    return dividend_df

In [13]:
temp = compute_indiv_collective_profit(profit, 50)
temp

(937.17999999999995, 937.18999999999994)

In [14]:
sum(temp)

1874.3699999999999

In [15]:
#Note that the collective proportionate patronage should be approximately 1.0,
#assuming 50% of net income is collective.
#It may be slightly greater or slightly less depending on whether
#the actual collective net profit is greater or the individual net
#profit is greater after rounding to nearest cents.
dividend_df = compute_dividends(patronage_df, profit)
dividend_df

,name,proportionate_patronage,dividend
member_id,,,
1,Alice,0.240185,225.10
2,Bob,0.295060,276.53
3,Candice,0.086323,80.90
4,Darwin,0.208926,195.80
5,Eve,0.130401,122.21
6,Federico,0.039105,36.65
0,CollectiveAcct,0.999989,937.18


In [16]:
#Check that the individual member dividends add up to about half the total profit,
#and that the individual allocations plus the collective allocations add up
#exactly (within floating point accuracy) to the total profit.
dividend_df.loc[1:6,'dividend'].sum(), dividend_df['dividend'].sum()

(937.18999999999994, 1874.3699999999999)

In [17]:
#See the floating point representation of the collective allocation
dividend_df.loc[0,'dividend']

937.17999999999995

In [18]:
#Check that the members' proportionate patronage ammounts add up to 1.0,
#and see the floating point representation of the collective proportionate patronage.
dividend_df.loc[1:6, 'proportionate_patronage'].sum(), dividend_df['proportionate_patronage'].sum()

(0.99999999999999978, 1.9999893298050553)

In [19]:
percent = 70
temp = compute_dividends(patronage_df, profit, percent_individual=percent)
print('ind: {}\ncol: {}'.format(*compute_indiv_collective_profit(profit, percent)))
print('tot:', temp['dividend'].sum())

#This ratio gives the percentages of the total profit rather than the percentages
#relative to the individual portion of the profit.
print(temp['proportionate_patronage']/temp['proportionate_patronage'].sum())
temp

ind: 1312.06
col: 562.31
tot: 1874.37
member_id
1    0.168130
2    0.206542
3    0.060426
4    0.146248
5    0.091281
6    0.027373
0    0.299999
Name: proportionate_patronage, dtype: float64


,name,proportionate_patronage,dividend
member_id,,,
1,Alice,0.240185,315.14
2,Bob,0.295060,387.14
3,Candice,0.086323,113.26
4,Darwin,0.208926,274.12
5,Eve,0.130401,171.09
6,Federico,0.039105,51.31
0,CollectiveAcct,0.428570,562.31


## Write a function to compute cash payouts and written notices of allocation 

In [20]:
dividend_df.drop(index=0, columns='proportionate_patronage')

,name,dividend
member_id,,
1,Alice,225.10
2,Bob,276.53
3,Candice,80.90
4,Darwin,195.80
5,Eve,122.21
6,Federico,36.65


In [28]:
def compute_notice_amount(dividend, first_year_percent, n_years):
    if dividend >= 0:
        #If dividend is positive, compute the amount to be paid out every year after the first year.
        return np.round(dividend*(100-first_year_percent)*0.01/(n_years-1),2)
    else:
        #If dividend is negative, distribute evenly over all n_years years.
        return np.round(dividend/n_years,2)

def compute_allocations(dividend_df, year, first_year_percent=50, n_years=3):
    """Computes allocations over next n_years years after the dividend year.
    """
    allocation_df = dividend_df.drop(index=0, columns='proportionate_patronage')
    
    #Compute the amount for written notices of allocation after first year
    notice_amounts = dividend_df['dividend'].apply(
        lambda dividend: compute_notice_amount(dividend, first_year_percent, n_years))
    
    #Compute first year amount
    allocation_df[str(year+1)] = np.round(dividend_df['dividend'] - (n_years-1)*notice_amounts, 2)
    
    for y in range(year+2, year+n_years+1):
        allocation_df[str(y)] = notice_amounts
        
    allocation_df.rename(columns={'dividend': str(year)+'_dividend'}, inplace=True)
        
    return allocation_df

In [32]:
temp = compute_allocations(dividend_df, 2017, first_year_percent=40, n_years=3)
temp

,name,2017_dividend,2018,2019,2020
member_id,,,,,
1,Alice,225.10,90.04,67.53,67.53
2,Bob,276.53,110.61,82.96,82.96
3,Candice,80.90,32.36,24.27,24.27
4,Darwin,195.80,78.32,58.74,58.74
5,Eve,122.21,48.89,36.66,36.66
6,Federico,36.65,14.65,11.00,11.00


In [35]:
temp.loc[:,'2018':].sum(axis=1)

member_id
1    225.10
2    276.53
3     80.90
4    195.80
5    122.21
6     36.65
dtype: float64

In [36]:
temp.loc[:,'2018']/temp['2017_dividend'].values

member_id
1    0.400000
2    0.399993
3    0.400000
4    0.400000
5    0.400049
6    0.399727
Name: 2018, dtype: float64

In [37]:
temp.loc[:,'2018':].divide(temp['2017_dividend'], axis='index')

,2018,2019,2020
member_id,,,
1,0.400000,0.300000,0.300000
2,0.399993,0.300004,0.300004
3,0.400000,0.300000,0.300000
4,0.400000,0.300000,0.300000
5,0.400049,0.299975,0.299975
6,0.399727,0.300136,0.300136


In [26]:
#possible schema for table to store written notices of allocation:
#primary_key:(member_id, year_issued, year_due), amount

#We'll also need to keep track of internal capital accounts:
#Balance = (member equity) + (preferred shares) + (interest?) + (written notices of allocation)
#All these should be available in Gnucash, so we simply need to sum them up for each member.